In [218]:
import json 
import random
from sklearn.metrics import f1_score
from printers import pretty_print_conl
from sklearn.metrics import f1_score
from __future__ import division
from treeops import bfs
lns = [json.loads(_) for _ in open("dev.jsonl")]

In [224]:
def heuristic_extract(jdoc):
    '''
    return the lowest vertex in the tree that contains the query terms
    '''
    from_root = [_['dependent'] for _ in jdoc["basicDependencies"] if _['governor'] == 0][0]
    best = from_root
    def tok_is_verb(vertex):
        gov = [o["pos"][0] for o in jdoc["tokens"] if o["index"] == v][0]
        return gov[0].lower() == "v"
    for v in get_walk_from_root(jdoc):  # bfs
        children = dfs(g=jdoc, hop_s=v, D=[])
        # the verb heuristic is b/c the min governing tree is often just Q itself
        if all(i in children for i in jdoc["q"]) and tok_is_verb(v):
            best = v
    return best

def get_path_to_root(v, jdoc):
    def get_parent(v):
        for _ in jdoc["basicDependencies"]:
            if _["dependent"] == v:
                return _["governor"]
        return _["governor"]
    out = [v]
    parent = get_parent(v)
    while parent != 0:
        v = parent
        out.append(parent)
        parent = get_parent(v)
    return out

def min_tree_to_root(jdoc):
    return {i for q in jdoc["q"] for i in get_path_to_root(q, jdoc)}

def len_tree(tree, jdoc):
    return sum(len(o['word']) for o in jdoc["tokens"] if o["index"] in tree)


def get_options(tree, jdoc):
    optionsd = {o["dependent"] for o in jdoc["basicDependencies"] if o["governor"] in tree and o["dependent"] not in tree}
    optionsg = {o["governor"] for o in jdoc["basicDependencies"] if o["dependent"] in tree and o["governor"] not in tree}
    return optionsd | optionsg
    
def append_at_random(tree, jdoc):
    s = get_options(tree, jdoc)
    added = random.sample(s, 1)[0]
    assert added not in tree
    tree.add(added)

def bottom_up_compression_random(jdoc):
    tree = min_tree_to_root(jdoc=jdoc)
    while len_tree(tree=tree, jdoc=jdoc) < jdoc["r"]:
        try:
            append_at_random(tree, jdoc)
        except ValueError: # it is possible to back into a corner where there are no V left to add.
                           # in these cases, you cant make compression longer and should just stop
            return tree
    return tree

def print_gold(jdoc):
    gold = jdoc["compression_indexes"]
    print " ".join([_["word"] for _ in jdoc["tokens"] if _["index"] in gold])

def print_tree(tree, jdoc):
    tk = [_["word"] for _ in jdoc["tokens"] if _["index"] in tree]
    print " ".join(tk)
    
def get_f1(predicted, jdoc):
    original_ixs = [_["index"] for _ in jdoc["tokens"]]
    y_true = [_ in jdoc['compression_indexes'] for _ in original_ixs]
    y_pred = [_ in predicted for _ in original_ixs]
    return f1_score(y_true=y_true, y_pred=y_pred)



In [225]:
tot = 0
for sentence in lns:
    predicted = bottom_up_compression_random(sentence)
    tot += get_f1(predicted, sentence)
tot/len(lns)
predicted, get_f1(predicted, sentence)

({5, 9, 10, 14, 15, 16}, 0.54545454545454541)

In [161]:
train = [json.loads(_) for _ in open("mini.train.jsonl")]

In [226]:
from collections import defaultdict

dep_counter = defaultdict(list)

for _ in train:
    toks = [i for i in _["tokens"] if i["index"] in _["compression_indexes"] + [0]]
    _["tokens"] = toks
    #print len(toks)
    for t in _["tokens"]:
        gov = [d["dep"] for d in _["basicDependencies"] if d["dependent"] == t["index"]]
        assert len(gov) == 1
        gov = gov[0]
        dep = [d["dep"] for d in _["basicDependencies"] if d["governor"] == t["index"]]
        for d in dep:
            dep_counter[gov].append(d)

from collections import Counter

dep_probs = defaultdict()
for d in dep_counter:
    c = Counter(dep_counter[d])
    c = {k:v/sum(c.values()) for k,v in c.items()}
    dep_probs[d] = c
    
print dep_probs['root']

{u'cc': 0.0299625468164794, u'nmod:tmod': 0.02247191011235955, u'compound:prt': 0.003745318352059925, u'nsubjpass': 0.011235955056179775, u'conj': 0.0299625468164794, u'dobj': 0.04119850187265917, u'neg': 0.00749063670411985, u'mark': 0.003745318352059925, u'auxpass': 0.00749063670411985, u'advcl': 0.018726591760299626, u'aux': 0.03745318352059925, u'parataxis': 0.0149812734082397, u'xcomp': 0.0299625468164794, u'nsubj': 0.09363295880149813, u'nummod': 0.00749063670411985, u'advmod': 0.018726591760299626, u'punct': 0.27340823970037453, u'compound': 0.06741573033707865, u'ccomp': 0.018726591760299626, u'nmod:poss': 0.0149812734082397, u'case': 0.018726591760299626, u'cop': 0.011235955056179775, u'dep': 0.018726591760299626, u'appos': 0.00749063670411985, u'det': 0.0299625468164794, u'nmod': 0.08239700374531835, u'amod': 0.0299625468164794, u'acl:relcl': 0.02247191011235955, u'root': 0.003745318352059925, u'acl': 0.02247191011235955}


In [268]:
sentence = train[0] 
tree = min_tree_to_root(jdoc=sentence)
q_by_prob = []

def add_children_to_q(vx, q, sentence):
    '''add a vertexes children to a queue, sort by prob'''
    children = [d for d in sentence['basicDependencies'] if d["governor"] == vx if d["dep"] not in ["punct"]]
    governor = [d for d in sentence['basicDependencies'] if d["dependent"] == vx][0]
    for c in children:
        print "adding", c["dependent"]
        c["prob"] = dep_probs[governor["dep"]][c["dep"]]
        q.append(c)
    q.sort(key=lambda x:x["prob"], reverse=True)

def remove_from_q(vx, Q, sentence):
    '''add a vertexes children to a queue, sort by prob'''
    print "deleting {}".format(vx)
    for ino, i in enumerate(Q):
        if i["dependent"] == vx:
            del Q[ino]
            break

for item in tree:
    add_children_to_q(item, q_by_prob, sentence)


while len_tree(tree=tree, jdoc=jdoc) < jdoc["r"]:
    try:
        tree.add(q_by_prob[0]["dependent"])
        add_children_to_q(q_by_prob[0]["dependent"], q_by_prob, jdoc)
        remove_from_q(q_by_prob[0]["dependent"], q_by_prob, sentence)
    except ValueError: # it is possible to back into a corner where there are no V left to add.
                       # in these cases, you cant make compression longer and should just stop
        pass


adding 1
adding 3
adding 6
adding 5
adding 7
adding 12
deleting 5
> <ipython-input-268-73594752374d>(22)remove_from_q()
-> break
(Pdb) Q
[{u'dependentGloss': u'Butler', u'dep': u'nsubj', u'governorGloss': u'said', u'governor': 4, u'dependent': 3, 'prob': 0.1569045964459446}, {u'dependentGloss': u'Favre', u'dep': u'dobj', u'governorGloss': u'pities', u'governor': 6, u'dependent': 7, 'prob': 0.10552242675654654}, {u'dependentGloss': u'find', u'dep': u'advcl', u'governorGloss': u'pities', u'governor': 6, u'dependent': 12, 'prob': 0.039495289949010456}, {u'dependentGloss': u'Today', u'dep': u'nmod:tmod', u'governorGloss': u'said', u'governor': 4, u'dependent': 1, 'prob': 0.03129653529236573}, {u'dependentGloss': u'pities', u'dep': u'ccomp', u'governorGloss': u'said', u'governor': 4, u'dependent': 6, 'prob': 0.02089744862503723}]
(Pdb) vx
5


KeyboardInterrupt: 